# DATA SCIENCE JOBS: SALARIES AND TYPE OF WORKS MODELS

## Project information

In recent years, data science roles are more demanded, so it's important to understand how they are evolving in terms of positioning as job offers and their salaries.

In this analysis, I will take my first Exploratory Data Analysis (EDA) and visualization project using Python, where I explored data science salaries between 2020 and 2023. 

The "global salary index" dataset comes from the ai-jobs.net website for roles in the AI, ML, Data Science space based on internal data obtanided from surveys and jobs with open salaries.

This dataset is processed and updated on a weekly basis but I'll take the dataset updated on September 29th, 2023.

## Hypothesis

## Understanding and preprocessing the dataset

### Importing Required Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
sns.set_theme()

%matplotlib inline

### Read dataset

In [7]:
df = pd.read_csv('../data/raw/salaries.csv')
df.head(500)

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/salaries.csv'

### Dataset shape and information

In [ ]:
print("Number of rows and columns in the dataset:",df.shape)

In [ ]:
# Information about the dataset including the index dtype and columns, non-null values and memory usage
      
df.info()

In [ ]:
# Generate descriptive statistics
      
df.describe()

In [ ]:
# Exploring column names of the dataframe

print("The column names of the dataset are::\n\n",df.columns)

### Missing values

In [ ]:
# Exploring if the dataset contains missing values

df.isnull().sum()/len(df)

In [ ]:
print("Number of missing data in the dataset:",df.isnull().sum().sum())

There is no missing data so it will not be necessary to delete any columns

### Unique values per column

In [ ]:
# Exploring unique values per each column of the dataset

print("Number of unique values in columns:\n\n", df.nunique())

With this exploration we can highlight some columns with many unique values, it will be important to analyze if we can work with these data or we will have to group them to reduce the size of the dataset.

- Job_title
- Salary
- Salary_in_usd
- Employee_residence
- Company location

### Overview data

________

## Data cleaning or transform it

Once we get all the overall information we can start working with the data and think about what changes we can make to clean up the data or transform it to ensure more meaningful consistency of certain values.

In [ ]:
# We need to create a copy of the original dataframe before doing any transformation on the original data
# We will call the new df "data_cleaning" to recognize the process we are working with

data_cleaning0 = df.copy()
data_cleaning0.head(2)

We will have to divide the dataset into numerical and categorical columns.

In [ ]:
# Before starting cleaning the numerical data we will need to transform the column "remote_ratio" into an object column

data_cleaning = data_cleaning0.copy()
data_cleaning['remote_ratio'] = data_cleaning['remote_ratio'].apply(str)

data_cleaning.dtypes

_______

## Cleaning numerical data

In [ ]:
# Create new DataFrame with numericals columns

numericals = data_cleaning.select_dtypes(np.number)
numericals.head()

In [ ]:
# Check values dtypes

numericals.dtypes

In [ ]:
# Count number of distinct elements in the numericals DataFrame 

numericals.nunique()

### "Work_year"

In [ ]:
# Check the unique values
numericals["work_year"].value_counts()

In [ ]:
# plot
fig, ax = plt.subplots()
sns.lineplot(x=numericals["work_year"].value_counts().index, y=numericals["work_year"].value_counts().values, ax = ax)
ax.set_ylabel("Number of jobs posted")
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(1))

### "Salary"

What we can take from this analysis is that there are two columns that have a number of values well above the rest, the salaries. In this case the data is duplicated because we have the same salary/job title in different currencies.

As the analysis will not be focused on wage differentials between countries it's better to focus the analysis with the same currency for all job titles, in this case we have a column with salaries in USD and we will use it for clearer visibility.

As a consequence, we will remove the "salary" column and change the name of the "salary_in_usd" column to "salary".

- "salary"
- "salary_in_usd"

In [ ]:
# Drop "salary" column

numericals.drop(labels=["salary"], axis=1, inplace=True)
numericals.rename(columns={"salary_in_usd": "salary"}, inplace=True)

numericals.head()

## Qualitative approaches to detect outliers

Technique to identify uni-dimensional outliers is to create a boxplot and to see if
there are isolated dots quite far away from the wiskers.

In [ ]:
#work_year

fig, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios":(.15, .85)})
sns.boxplot(data=numericals, x="work_year", ax=ax_box) #display boxplot
sns.histplot(data=numericals, x="work_year", ax=ax_hist) #hist of the same column
ax.set_ylabel("Number of jobs posted")
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(1))

plt.show() 

In [ ]:
# Compute the skewness
(numericals['work_year'].skew() > 2) or (numericals['work_year'].skew() < -2), numericals['work_year'].skew()

We have a negative skew on the "work_year" column.
That means a longer tail on the left side of the distribution, or in other words, is the direction or weight of the distribution. 

We have more values from 2023 on our dataset, and just few from 2020 (outliers).

In [ ]:
#salary

fig, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios":(.15, .85)})
sns.boxplot(data=numericals, x="salary", ax=ax_box) #display boxplot
sns.histplot(data=numericals, x="salary", ax=ax_hist) #hist of the same column
plt.show() 

In [ ]:
# Compute the skewness
(numericals['salary'].skew() > 2) or (numericals['salary'].skew() < -2), numericals['salary'].skew()

For the "salary" column we have a little positive skew.
That means a longer tail on the right side of the distribution.
Just few salaries are between 30-40K.

_____

## Cleaning categorical data

Now, we will proceed to pre-processing the categorical data (clean and transformed).

Transform categorical data (encoded) https://www.datacamp.com/tutorial/categorical-data

Confirm if it's necessary or not

In [ ]:
# Create new DataFrame with categoricals columns

categoricals = data_cleaning.select_dtypes([object])
categoricals.head()

In [ ]:
categoricals.value_counts()

Looking at the values included in each column we can see that some of them may be confusing, the current values  don't help us to understand the real meaning.

The columns affected are:

- "experience_level"
- "employment_type"
- "employee_residence"
- "remote_ratio"
- "company_location"
- "company_size"

We will need to evaluate individually to define if we need to replace or group them.

### "Experience_level"

In [ ]:
# Check the unique values
categoricals["experience_level"].value_counts()

In [ ]:
"""We know that the real meanings per each value are:
    SE - Senior
    MI - Mid
    EN - Entry
    EX - Executive
    
    We need to apply a <.replace> to replace each value with its real meaning """

replace_cat = categoricals.copy()
replace_cat.experience_level.replace(['EN','MI','SE', 'EX'], ['entry', 'mid', 'senior', 'executive'], inplace = True)

In [ ]:
# Check if the substitution has been applied correctly

replace_cat["experience_level"].value_counts()

In [ ]:
#  create a plot with Tableau

### "Employment_type"

In [ ]:
# Check the unique values
categoricals["employment_type"].value_counts()

In [ ]:
"""We know that the real meanings per each value are:
    FT - Full_time
    CT - Contract
    PT - Part_Time
    FL - Freelance
    
    We need to apply a <.replace> to replace each value with its real meaning """

replace_cat.employment_type.replace(['FT','CT' , 'PT', 'FL'], ['full_time', 'contract', 'part_Time', 'freelance'], inplace = True)

In [ ]:
replace_cat['employment_type'].value_counts()

In [ ]:
# plot



### "Remote_ratio"

In [ ]:
# Check the unique values
categoricals["remote_ratio"].value_counts()

In [ ]:
"""We know that the real meanings per each value are:
    0 - No_remote/On_site
    50 - Hybrid
    100 - Remote
    
    We need to apply a <.replace> to replace each value with its real meaning """

replace_cat.remote_ratio.replace(['0','50','100'], ['on_site', 'hybrid', 'remote'], inplace = True)

In [ ]:
replace_cat['remote_ratio'].value_counts()

In [ ]:
 # create a plot

### "Company_size"

In [ ]:
# Check the unique values
categoricals["company_size"].value_counts()

In [ ]:
"""We know that the real meanings per each value are:
    S - small
    M - medium
    L - marge
    
    We need to apply a <.replace> to replace each value with its real meaning """

replace_cat.company_size.replace(['S','M','L'], ['small', 'medium', 'large'], inplace = True)

In [ ]:
replace_cat["company_size"].value_counts()

In [ ]:
 # create a plot

### "Salary_currency"

As mentioned above, the salary analysis will be done only with USD currency, that is why we can delete the column "salary_currency" to avoid having data that could create interferences in our analysis.


In [ ]:
# Check the unique values
categoricals["salary_currency"].value_counts()

In [ ]:
replace_cat.drop(["salary_currency"], axis=1, inplace=True)

In [ ]:
#Check if the column has been dropped correctly

replace_cat.head()

In [ ]:
 # create a plot

### "Employee_residence" & "Company_location"

In [ ]:
print("Number of unique values in employee_residence column is:\n\n", categoricals["employee_residence"].nunique())

In [ ]:
print("Number of unique values in company_locations column is:\n\n", categoricals["company_location"].nunique())

We want to replace the codes for locations or country names

Python has a function called "pycountry" (https://pypi.org/project/pycountry/) that provides the ISO databases for countries and others. We know that the <codes> in our database are encoded based on the "ISO" because it's specified in the "Legend" in the web from where we have extracted the database (https://ai-jobs.net/salaries/).

In [ ]:
"""We want to replace the locations and countries residence codes 
for employees and companies to reduce the values size.

We will need to use a For Loop to go from code to code and create a new list with the new value names.

We will need to use the library "pycountry":

#Convert country code ISO 3166-1 alpha-2 to country name:
#country_alpha2_to_country_name(cn_name_format="default") 
"""

from pycountry import countries

employee_country = []
company_country = []

for country_code in replace_cat.employee_residence:
   employee_country.append(pycountry.countries.get(alpha_2=country_code).name)

for country_code in replace_cat.company_location:
    company_country.append(pycountry.countries.get(alpha_2=country_code).name)

replace_cat['employee_residence'] = employee_country
replace_cat['company_location'] = company_country


#https://snyk.io/advisor/python/pycountry/functions/pycountry.countries.get

In [ ]:
# Check if the replacement from country code to coutnry names has ben placed correctly

replace_cat.head()

In [ ]:
# create a plot

### "Job_title"

In [ ]:
# Check the unique values
categoricals["job_title"].value_counts()

In [ ]:
# Histograma
fig= plt.subplots
sns.histplot(y="job_title", data=replace_cat);
ax.set_title("Jobs per work year")